In [1]:
%pylab nbagg
from tvb.simulator.lab import *
import os.path
from matplotlib import colors, cm
import time
import scipy.signal as sig
import scipy.io as sio
%load_ext autoreload
%autoreload 2

Populating the interactive namespace from numpy and matplotlib


In [2]:
nez=0
npz=6
Gez = [[],[9],[9,6],[9,6,13]]
Gpz = [[5,6,7,11,13,27,34,83],[5,6,7,11,13,27,34,83],[5,6,7,11,13,27,34,83],[5,6,7,11,28,27,34,83]]
project_dir = "./data"
timeseries_dir='./Simulated'
additionname='SN0_'

In [ ]:
# Patient specific modifications
def mln_simulation(nez,npz):
    ez=Gez[nez]
    pz=Gpz[nez][:npz]



    con = connectivity.Connectivity.from_file(os.path.join(project_dir, "connectivity.zip"))
    con.speed = np.inf
    # normalize
    con.weights = con.weights/np.max(con.weights)
    num_regions = len(con.region_labels)

    #figure()
    image = con.weights
    norm = colors.LogNorm(1e-7, image.max()) #, clip='True')
    #imshow(image, norm=norm, cmap=cm.jet)
    #colorbar()
    #max(con.weights[con.weights != 0])

    epileptors = models.Epileptor(variables_of_interest=['x1', 'y1', 'z', 'x2', 'y2', 'g', 'x2 - x1'])
    epileptors.r = 0.0001
    epileptors.Ks = np.ones(num_regions)*(-1.0)*20.0
3

    x0ez=-1.8
    x0pz=-2.05
    x0num=-2.3
    epileptors.x0 = np.ones(num_regions)*x0num

    epileptors.x0[ez] = x0ez
    epileptors.x0[pz] = x0pz

    epileptors

    coupl = coupling.Difference(a=1.)

    #hiss = noise.Additive(nsig = numpy.array([0.01, 0.01, 0., 0.00015, 0.00015, 0.]))
    hiss = noise.Additive(nsig = numpy.array([0.0, 0.0, 0., 0., 0., 0.]))
    heunint = integrators.HeunStochastic(dt=0.04, noise=hiss)
    #heunintdet = integrators.HeunDeterministic(dt=0.04)

    mon_tavg = monitors.TemporalAverage(period=1.0)
    mon_SEEG = monitors.iEEG.from_file(sensors_fname=os.path.join(project_dir, "seeg.txt"),
                                       projection_fname=os.path.join(project_dir, "gain_inv-square.txt"),
                                       rm_f_name=os.path.join(project_dir, "regionMapping_16k_76.txt"),
                                       period=1.0,
                                       variables_of_interest=[6]
                                       )
    num_contacts = mon_SEEG.sensors.labels.size

    con.cortical[:] = True     # To avoid adding analytical gain matrix for subcortical sources

    sim = simulator.Simulator(model=epileptors,
                              connectivity=con,
                              coupling=coupl,
                              conduction_speed=np.inf,                          
                              integrator=heunint,#heunintdet,#
                              monitors=[mon_tavg, mon_SEEG])

    sim.configure()

    (ttavg, tavg), (tseeg, seeg) = sim.run(simulation_length=10000)

    # Normalize the time series to have nice plots
    tavgn = tavg/(np.max(tavg, 0) - np.min(tavg, 0))
    seegn = seeg/(np.max(seeg, 0) - np.min(seeg, 0))
    seegn = seegn - np.mean(seegn, 0)

    b, a = sig.butter(2, 0.1, btype='highpass', output='ba')
    #seegf = sig.filtfilt(B, A, seegn)
    seegf = np.zeros(seegn.shape)
    for i in range(num_contacts):
        seegf[:, 0, i, 0] = sig.filtfilt(b, a, seeg[:, 0, i, 0])

    #Plot raw time series
    figure(figsize=(9,10))

    indf = 0
    indt = -1

    regf = 0
    regt = 84

    plot(ttavg[indf:indt], tavg[indf:indt, 6, regf:regt, 0]/4 + np.r_[regf:regt], 'r')
    yticks(np.r_[regf:regt], con.region_labels[regf:regt])
    title("Epileptors time series")
    tight_layout()
    show()
    savefig(os.path.join(timeseries_dir, additionname+'NodetimeSeries_Nez_'+str(nez)+'_Npz_'+str(npz)+'.png'))

    figure(figsize=(10,20))
    plot(tseeg[:], seegn[:, 0, :, 0] + np.r_[:num_contacts])
    yticks(np.r_[:num_contacts], mon_SEEG.sensors.labels[:])
    title("SEEG")
    tight_layout()
    savefig(os.path.join(timeseries_dir, additionname+'SeegtimeSeries_Nez_'+str(nez)+'_Npz_'+str(npz)+'.png'))

    figure(figsize=(10,20))
    plot(tseeg[:], (seegn[:, 0, 1:num_contacts, 0] - seegn[:, 0, 0:num_contacts-1, 0]) + np.r_[:num_contacts-1])
    yticks(np.r_[:num_contacts], mon_SEEG.sensors.labels[:])
    title("SEEG")
    tight_layout()
    savefig(os.path.join(timeseries_dir, additionname+'bipSeegtimeSeries_Nez_'+str(nez)+'_Npz_'+str(npz)+'.png'))

    figure(figsize=(10, 6))

    electrodes = [("FCA'", 7), ("GL'", 7), ("CU'", 6), ("PP'", 1),
                  ("PI'", 5), ("GC'", 8), ("PFG'", 10),
                  ("OT'", 5), ("GPH'", 6), ("PFG", 10)]


    for i, (el, num) in enumerate(electrodes):
        ind = np.where(mon_SEEG.sensors.labels == el + str(num))[0][0]
        plot(tseeg[:], (seegn[:, 0, ind, 0] - seegn[:, 0, ind - 1, 0])/0.5 + i)

    labels = [el[0] + str(el[1]) + "-" + str(el[1] - 1) for el in electrodes]
    yticks(np.r_[:len(electrodes)], labels)
    tight_layout()
    savefig(os.path.join(timeseries_dir, additionname+'bipChoosenSeegtimeSeries_Nez_'+str(nez)+'_Npz_'+str(npz)+'.png'))

    a_dict={'tsSEEG':seegn,
           'tpSEEG':tseeg,
           'tpNodes':ttavg,
            'tsNodes':tavg,
            'x0':{'x0ez':x0ez,'x0pz':x0pz,'x0num':x0num},
            'ez':con.region_labels[ez],
            'pz':con.region_labels[pz],
            'Interestedbip':labels,
           }

    sio.savemat(os.path.join(timeseries_dir, additionname+'timeSeries_Nez_'+str(nez)+'_Npz_'+str(npz)+'.mat'),{'Data':a_dict})

In [ ]:
for nez in np.arange(0,4).astype(int):
    for npz in np.arange(1,8).astype(int):
        mln_simulation(nez,npz)
